<a href="https://colab.research.google.com/github/cengaiyeung/20MA573/blob/master/src/hw03_part3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- Prove the following facts: Supose $f$ is a function satisfying
  - $f(0) = f_{min},$ and $\lim_{x\to \infty}f(x) =  f_{max}$
  - $f$ is continuous
  - $f$ is strictly increasing
  
  then, for any $p\in  (f_{min}, f_{max})$, 
  - there exists unique $\hat \sigma$, such that $f(\hat \sigma) = p$ and  
  $$\hat \sigma = argmin_{\sigma\in (0,\infty)} | f(\sigma) - p|.$$

- Now we denote by $f(\sigma)$ the BSM put price with the following parameters:
  - vol_ratio = $\sigma$; spot_price = 100.; drift_ratio = .0475; strike = 110.; maturity = 1.
  
  Answer the following questions:
  
  - What is $f_{min}$ and $f_{max}$?

  - Is $f$ strictly increasing on $(0,\infty)$? Justify your answer.

  - If the market put price is $10$, then what's the implied volatility?

In [0]:
import numpy as np
import scipy.stats as ss
import matplotlib.pyplot as plt
from pylab import plt
plt.style.use('seaborn')
%matplotlib inline

'''=========
option class init
=========='''
class VanillaOption:
    def __init__(
        self,
        otype = 1, # 1: 'call'
                  # -1: 'put'
        strike = 110.,
        maturity = 1.,
        market_price = 10.):
      self.otype = otype
      self.strike = strike
      self.maturity = maturity
      self.market_price = market_price #this will be used for calibration
      
        
    def payoff(self, s): #s: excercise price
      otype = self.otype
      k = self.strike
      maturity = self.maturity
      return max([0, (s - k)*otype])
'''============
Gbm class
============='''

class Gbm:
    def __init__(self,
                 init_state = 100.,
                 drift_ratio = .0475,
                 vol_ratio = .2
                ):
        self.init_state = init_state
        self.drift_ratio = drift_ratio
        self.vol_ratio = vol_ratio

'''========
Black-Scholes-Merton formula. 
=========='''

def bsm_price(self, vanilla_option):
    s0 = self.init_state
    sigma = self.vol_ratio
    r = self.drift_ratio
    
    otype = vanilla_option.otype
    k = vanilla_option.strike
    maturity = vanilla_option.maturity
    
    d1 = (np.log(s0 / k) + (r + 0.5 * sigma ** 2) 
          * maturity) / (sigma * np.sqrt(maturity))
    d2 = d1 - sigma * np.sqrt(maturity)
    
    return (otype * s0 * ss.norm.cdf(otype * d1) #line break needs parenthesis
            - otype * np.exp(-r * maturity) * k * ss.norm.cdf(otype * d2))

Gbm.bsm_price = bsm_price

In [0]:
gbm1 = Gbm(
    init_state = 100., 
    drift_ratio = .0475,
    vol_ratio = .2)
option1 = VanillaOption(
    otype = -1,
    strike = 110.,                
    maturity = 1.
)    

In [3]:
## define an error function
def error_function(vol, gbm, option):
  gbm.vol_ratio = vol
  return abs(option.market_price - gbm.bsm_price(option))


## define a method to seek for an implied volatility
import scipy.optimize as so
def implied_volatility(gbm, option):
  init_vol = .1 #initial guess
  return so.fmin(error_function, init_vol, 
                 args = (gbm, option), disp = 0)[0]

## test the implied_vol by reversing bsm_formula example in the above
option1.market_price = 10

print('>>>>>>>>implied volatility is ' + 
     str(implied_volatility(gbm1, option1)))

>>>>>>>>implied volatility is 0.17867187500000026


- Find its implied volatility with the following parameters:
  - BSM call price is 10.;  spot_price = 100.; drift_ratio = .0475; strike = 110.; maturity = 1.

In [0]:
gbm2 = Gbm(
    init_state = 100., 
    drift_ratio = .0475,
    vol_ratio = .2)
option2 = VanillaOption(
    otype = 1,
    strike = 110.,                
    maturity = 1.
)

In [5]:
## test the implied_vol by reversing bsm_formula example in the above
option2.market_price = 10

print('>>>>>>>>implied volatility is ' + 
     str(implied_volatility(gbm2, option2)))

>>>>>>>>implied volatility is 0.3020312500000007
